## Things to represent
- The actual utility functions
- 

In [2]:
import pycid
import numpy as np
from pycid.core.mechanised_graph import MechanisedGraph

c:\Users\jhr\Anaconda3\envs\pycid\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def utility_function_follower(d1: tuple, d2: int) -> float:
    """From GT slides"""
    R = 2
    P = 1
    udef = 1 - d1[d2-1] * R + d1[d2-1] * P
    return udef

def utility_function_leader(d1: tuple, d2: int) -> float:
    R = 2
    P = 1
    udef = 1 - d1[d2-1] * P + d1[d2-1] * R
    return udef


In [4]:


# Simple MACID setup
macid = pycid.MACID(
    [("D1", "U1"), ("D1", "U2"), ("D2", "U1"), ("D2", "U2")],
    agent_decisions={1: ["D1"], 2: ["D2"]},
    agent_utilities={1: ["U1"], 2: ["U2"]},
)

# Discretise domains to fit with PyCID (It is not possible to use continuous domains in pygambit, right?)
PRECISION = 0.2
CONSTRAINT = 1
d1_domain = [(a, b, c) for a in np.arange(0, 1.1, PRECISION) for b in np.arange(0, 1.1, PRECISION) for c in np.arange(0, 1.1, PRECISION) if a + b + c == CONSTRAINT]
d2_domain = list(range(1, 4))

macid.add_cpds(
    D1 = d1_domain,
    D2 = d2_domain,
    U1 = lambda D1, D2: utility_function_leader(D1, D2),
    U2 = lambda D1, D2: utility_function_follower(D1, D2),
)


mech = MechanisedGraph(macid)

In [5]:
# solve for all nash equilibria
nash_eq = macid.get_ne(solver="enummixed")

C:\Users\jhr\pycid\pycid\core\macid_base.py:59: UserWarning: adding DecisionDomain to non-decision node D1
  warn(f"adding DecisionDomain to non-decision node {variable}")
C:\Users\jhr\pycid\pycid\core\macid_base.py:59: UserWarning: adding DecisionDomain to non-decision node D2
  warn(f"adding DecisionDomain to non-decision node {variable}")


In [7]:
from pycid.core.cpd import StochasticFunctionCPD

def any_high_prob(CPD: StochasticFunctionCPD, threshold=0.9) -> bool:
    return any(probability for probability in CPD.get_values() if probability > threshold)

In [9]:
# process for finding stackelberg equilibrium: find the strategy (or meta-strategy) that has the highest probability and then commit to that. 
# An alternative approach would be to commit to the highest utility meta-strategy, but is that allowed? :))

high_probability_strategies = [CPD for CPD in nash_eq if any_high_prob(CPD[0], threshold=0.9)]

# find the strategy that has the highest probability
highest_probability_strategy = max(nash_eq, key=lambda CPD: max(CPD[0].get_values()))


In [11]:
highest_probability_strategy

[StochasticFunctionCPD<D1> 
 {}  ->  {(0.0, 0.0, 1.0): 0.0, (0.0, 0.2, 0.8): 0.0, (0.0, 0.4, 0.6000000000000001): 0.0, (0.0, 0.6000000000000001, 0.4): 0.0, (0.0, 0.8, 0.2): 0.0, (0.0, 1.0, 0.0): 0.0, (0.2, 0.0, 0.8): 0.0, (0.2, 0.2, 0.6000000000000001): 0.0, (0.2, 0.4, 0.4): 0.8333333333333335, (0.2, 0.6000000000000001, 0.2): 0.0, (0.2, 0.8, 0.0): 0.0, (0.4, 0.0, 0.6000000000000001): 0.0, (0.4, 0.2, 0.4): 0.0, (0.4, 0.4, 0.2): 0.0, (0.4, 0.6000000000000001, 0.0): 0.0, (0.6000000000000001, 0.0, 0.4): 0.0, (0.6000000000000001, 0.2, 0.2): 0.0, (0.6000000000000001, 0.4, 0.0): 0.0, (0.8, 0.0, 0.2): 0.0, (0.8, 0.2, 0.0): 0.0, (1.0, 0.0, 0.0): 0.16666666666666657},
 StochasticFunctionCPD<D2> 
 {}  ->  {1: 0.33333333333333326, 2: 0.3333333333333333, 3: 0.33333333333333354}]